In [1]:
from configuration import EC2_CONFIG, IAM_CONFIG, PROFILE, CODE_DEPLOY_CONFIG, IAM_CONFIG, S3_CONFIG, STS_CONFIG, CLOUD_WATCH_CONFIG
from configuration import REGION, AVAILABILITY_ZONE
from aws_service_utils import create_aws_service
from ec2_utils import create_vpc, create_subnet, create_security_group, create_key_pair, \
    lunch_ec2_instance, wait_instances_lunch, get_subnet_ids, start_ec2_instance, stop_ec2_instance

from iam_utils import get_role
from code_deploy_utils import create_application, create_deployment, create_deployment_group
from sts_utils import get_aws_user_account
from s3_utils import create_bucket, put_bucket_policy, upload_server_app_to_s3_bucket

In [2]:
ec2 = create_aws_service(service=EC2_CONFIG['service_name'],
                             region=REGION
                             )
iam = create_aws_service(IAM_CONFIG['service_name'],
                             region=REGION
)

code_deploy = create_aws_service(CODE_DEPLOY_CONFIG['service_name'],
                             region=REGION
)

s3 = create_aws_service(S3_CONFIG['service_name'],
                             region=REGION
)
sts = create_aws_service(STS_CONFIG['service_name'],
                             region=REGION
)


Service ec2 is created
Service iam is created
Service codedeploy is created
Service s3 is created
Service sts is created


In [3]:
vpc_id = create_vpc(ec2, cidr_block='10.0.0.0/16')
sec_group_id = create_security_group(ec2=ec2, vpc_id=vpc_id, group_name=EC2_CONFIG['security_group'])
key_id = create_key_pair(ec2=ec2, key_name=EC2_CONFIG['key_pair'])
image_id = EC2_CONFIG['ImageId']
subnet_id = create_subnet(ec2, vpc_id, cidr_block='10.0.4.0/24', availability_zone=AVAILABILITY_ZONE)


VPC created vpc-0e4fa9a38c1a4a62c
security group created sg-0c0bbef9914fc0bcc
Key pair created key_pair_lab2
Subnet id created subnet-04e1503c9a554e75a


In [4]:

worker = lunch_ec2_instance(
            ec2=ec2,
            image_id=image_id,
            instance_type=EC2_CONFIG['instance_type'],
            key_name=EC2_CONFIG['key_pair'],
            zone=AVAILABILITY_ZONE,
            profile=PROFILE,
            sec_group_ids=sec_group_id,
            subnet_id=subnet_id, 
            tags=EC2_CONFIG['worker']['TagSpecifications']
        )




#worker = 'i-036b3a2eabd9a6619'

Instance lunched i-0c2e39162e1349208


In [5]:
start_ec2_instance(ec2, worker)

Started EC2 instance with ID: i-0c2e39162e1349208


In [6]:
wait_instances_lunch(ec2=ec2, instances_ids=[worker])

In [ ]:
role_arn = get_role(iam, IAM_CONFIG['role'])

user_account = get_aws_user_account(sts)
bucket = create_bucket(s3, S3_CONFIG['worker']['bucket'])
put_bucket_policy(s3, S3_CONFIG['bucket_policy'], bucket, user_account, role_arn)
upload_server_app_to_s3_bucket(s3, bucket)

#bucket = "log8415-lab2-bucket1698972750"

In [8]:
worker_app_id = create_application(code_deploy=code_deploy, application_name=CODE_DEPLOY_CONFIG['worker']['application_name'])
#worker_app_id = '593fc06e-b677-4f00-9b13-ce0226afa12d'

Role: arn:aws:iam::175054333530:role/LabRole


AWS account : 175054333530


S3 bucket created: lab2-worker-bucket1699243795
Policy applied to lab2-worker-bucket1699243795


Flask app uploaded tolab2-worker-bucket1699243795
Application created: 593fc06e-b677-4f00-9b13-ce0226afa12d


In [9]:
# create the deployment group
worker_dep_group= create_deployment_group(
        code_deploy = code_deploy, service_role_arn=role_arn, app_name=CODE_DEPLOY_CONFIG['worker']['application_name'], 
        dep_group_name=CODE_DEPLOY_CONFIG['worker']['group_name'], tag_filters = CODE_DEPLOY_CONFIG['worker']['EC2TagFilters']
    )
    
'''
orchestrator_dep_group= create_deployment_group(
        code_deploy = code_deploy, service_role_arn=role_arn, app_name=CODE_DEPLOY_CONFIG['application_name'],
        dep_group_name=CODE_DEPLOY_CONFIG['group_name'], tag_filters = CODE_DEPLOY_CONFIG['tags']
    )
'''


Deploymenet group created eb0829e9-c161-4c94-ab2f-6416dc682775


"\norchestrator_dep_group= create_deployment_group(\n        code_deploy = code_deploy, service_role_arn=role_arn, app_name=CODE_DEPLOY_CONFIG['application_name'],\n        dep_group_name=CODE_DEPLOY_CONFIG['group_name'], tag_filters = CODE_DEPLOY_CONFIG['tags']\n    )\n"

In [10]:
#create_deployment(code_deploy, bucket, revision, deploy_group_name, app_name)
deployment_id_worker = create_deployment(
        code_deploy,
        bucket,
        'worker.zip',
        CODE_DEPLOY_CONFIG['Revision'], 
        CODE_DEPLOY_CONFIG['worker']['group_name'],
        CODE_DEPLOY_CONFIG['worker']['application_name']


    )

Application deployment launched d-GKDZQY1N2
